# Krzysztof Nalepa
## Sprawozdanie z ćwiczenia 1

### Zadanie 1
Wykorzystaj funkcję finfo z biblioteki numpy aby określić precyzję i zakres różnych typów danych reprezentujących liczby zmienno-przecinkowe. Wyniki badania przedstaw w postaci zestawienia tabelrycznego

In [29]:
import numpy as np

print('-------16 bit-------')
print('Min:      ', np.finfo(np.float16).min)
print('Max:       ', np.finfo(np.float16).max)
print('Precision: ', np.finfo(np.float16).precision)

print('-------32 bit-------')
print('Min:      ', np.finfo(np.float32).min)
print('Max:       ', np.finfo(np.float32).max)
print('Precision: ', np.finfo(np.float32).precision)

print('-------64 bit-------')
print('Min:      ', np.finfo(np.float64).min)
print('Max:       ', np.finfo(np.float64).max)
print('Precision: ', np.finfo(np.float64).precision)

print('-------128 bit-------')
print('Min:      ', np.finfo(np.float128).min)
print('Max:       ', np.finfo(np.float128).max)
print('Precision: ', np.finfo(np.float128).precision)

-------16 bit-------
Min:       -65500.0
Max:        65500.0
Precision:  3
-------32 bit-------
Min:       -3.4028235e+38
Max:        3.4028235e+38
Precision:  6
-------64 bit-------
Min:       -1.7976931348623157e+308
Max:        1.7976931348623157e+308
Precision:  15
-------128 bit-------
Min:       -1.189731495357231765e+4932
Max:        1.189731495357231765e+4932
Precision:  18


### Zadanie 2 
Napisz funkcje, która dla dowolnej liczby zmiennoprzecinkowej w precyzji 64-bitowej (w standardzie IEEE 754) przedstawi w wersji dziesiętnej i binarnej składniki liczby: znak, wykładnik i mantysa. Można wykorzystać między innymi bibliotekę bitstring. Zbadaj jak reprezentowaną są poszczególne składniki tego formatu: zakres, sposób kodowania liczby etc.

In [27]:
from bitstring import BitArray

def to_particles(float_number):
  float_array = BitArray(float=float_number, length=64)
  sign_bin = float_array.bin[0]
  exponent_bin = float_array.bin[1:12]
  mantissa_bin = float_array.bin[12:]

  if sign_bin == 1:
    sign_dec = "-"
  else:
    sign_dec = "+"
  
  exponent_dec = 0
  pow = 10
  for i in exponent_bin[0:]:
    if i == "1":
      exponent_dec += 2 ** pow
    pow = pow - 1
  
  pow = -1
  mantissa_dec = 0
  for i in mantissa_bin[0:]:
    if i == "1":
      mantissa_dec += 2 ** pow
    pow = pow - 1
  
  print(f'Binary representation: {sign_bin}, {exponent_bin}, {mantissa_bin}')
  print(f'Decimal representation: {sign_dec}, {exponent_dec}, {mantissa_dec}')

to_particles(12.25)

Binary representation: 0, 10000000010, 1000100000000000000000000000000000000000000000000000
Decimal representation: +, 1026, 0.53125


### Zadanie 3 
Wykorzystaj funkcję nextafter() z biblioteki numpy (lub inny sposób) i sprawdź rozdzielczość typu float (jakie mają wartości najbliższe sąsiadujące liczby float). Zaprezentuj wynik korzystając z funkcji z Zadania 2.<br><br>
Wiedząc że liczby zmiennoprzecinkowe są najgęściej rozłożone blisko 0 uznałem że rozdzielczością będzie odległość między 0 a najbliższą mu liczbą.

In [28]:
import numpy as np

number = np.nextafter(0,1)
print(f'Resolution: {number}')
to_particles(number)

Resolution: 5e-324
Binary representation: 0, 00000000000, 0000000000000000000000000000000000000000000000000001
Decimal representation: +, 0, 2.220446049250313e-16


### Zadanie 4
Porównać reprezentację bitową liczby 1/3 dla typów float o różnej precyzji: float16, float32, float64. Sprawdź co się dzieje gdy zmienna o mniejszej precyzji konwertowana jest do wyższej. 

In [54]:
import numpy as np
from bitstring import BitArray
float_16 = np.float16(1/3)
float_32 = np.float32(1/3)
float_64 = np.float64(1/3)

# Bitstring nie dopuszcza BitArray 16 bitowej, więc dla każdego floata użyłem 64 bitowej
print(f'1/3 in float 16: {BitArray(float=float_16, length=64).bin}')
print(f'1/3 in float 32: {BitArray(float=float_32, length=64).bin}')
print(f'1/3 in float 64: {BitArray(float=float_64, length=64).bin}')
float_16_converted32 = np.float32(float_16)
float_16_converted64 = np.float64(float_16)
float_32_converted64 = np.float64(float_32)
print(f'Conversion float 16 to float 32 in binary: {BitArray(float=float_16_converted32, length=64).bin}')
print("Float 16: ", float_16, ", float 16 converted to 32: ", float_16_converted32)
print(f'Conversion float 16 to float 64: {BitArray(float=float_16_converted64, length=64).bin}')
print("Float 16: ", float_32, ", float 16 converted to 32: ", float_16_converted64)
print(f'Conversion float 32 to float 64: {BitArray(float=float_32_converted64, length=64).bin}')
print("Float 16: ", float_32, ", float 16 converted to 32: ", float_32_converted64)


1/3 in float 16: 0011111111010101010101000000000000000000000000000000000000000000
1/3 in float 32: 0011111111010101010101010101010101100000000000000000000000000000
1/3 in float 64: 0011111111010101010101010101010101010101010101010101010101010101
Conversion float 16 to float 32 in binary: 0011111111010101010101000000000000000000000000000000000000000000
Float 16:  0.3333 , float 16 converted to 32:  0.33325195
Conversion float 16 to float 64: 0011111111010101010101000000000000000000000000000000000000000000
Float 16:  0.33333334 , float 16 converted to 32:  0.333251953125
Conversion float 32 to float 64: 0011111111010101010101010101010101100000000000000000000000000000
Float 16:  0.33333334 , float 16 converted to 32:  0.3333333432674408


Jak widać konwersja z liczby o mniejszej precyzji do większej jest niedokładna ponieważ dodaje do mantysy 0 tak aby wypełnić brakujące bity. Jest to szcególnie zauważalne dla takich liczb jak 1/3 które są okresowe zapisane w systemie binarnym

### Zadanie 5
Sprawdź czym jest postać znormalizowana liczby typu float. Poniższy program generuje ciąg liczb. Wyświetl ich postać binarną i sprawdź czy są w postaci znormalizowanej (napisz stosowny kod, który to sprawdza). 

Zgodnie ze standardem IEEE 754 32 bitowe liczby znormalizowane są to liczby których cecha jest zawarta w przedziale (0,255). W naszym przypadku liczby znormalizowane będą to wszystkie liczby których cecha będzie równa 0.

In [95]:
from bitstring import BitArray

def check_if_zero(exponent):
  for number in exponent:
    if (number == "1"):
      return False
  return True


print("Exponent         Mantissa         is_normalized")
a=np.float32(1.1)
for i in range(0, 149):
    a=a/np.float32(2.0)
    a_array = BitArray(float=a, length = 32)
    print(a_array.bin[1:9], a_array.bin[9:], not check_if_zero(a_array.bin[1:9]))

Exponent         Mantissa         is_normalized
01111110 00011001100110011001101 True
01111101 00011001100110011001101 True
01111100 00011001100110011001101 True
01111011 00011001100110011001101 True
01111010 00011001100110011001101 True
01111001 00011001100110011001101 True
01111000 00011001100110011001101 True
01110111 00011001100110011001101 True
01110110 00011001100110011001101 True
01110101 00011001100110011001101 True
01110100 00011001100110011001101 True
01110011 00011001100110011001101 True
01110010 00011001100110011001101 True
01110001 00011001100110011001101 True
01110000 00011001100110011001101 True
01101111 00011001100110011001101 True
01101110 00011001100110011001101 True
01101101 00011001100110011001101 True
01101100 00011001100110011001101 True
01101011 00011001100110011001101 True
01101010 00011001100110011001101 True
01101001 00011001100110011001101 True
01101000 00011001100110011001101 True
01100111 00011001100110011001101 True
01100110 00011001100110011001101 True
01

### Zadanie 6
Wyjaśnij dlaczego poniższy program nie działa prawidłowo. Sprawdź co się stanie gdy użyjemy typów o mniejszej precyzji. Zaproponuj inny sposób sumowania, który będzie dawał dokładne wyniki. 

In [0]:
N = 100000000
C = 0.09531258654533566;

sum = 0.0 
for i in range(0, N):
    sum += C
    
print(sum)
print(N*C)

Powyższy program nie działa prawidłowo ponieważ próbujemy do dużej liczby zmiennoprzecinkowej dodać małą liczbe zmiennoprzecinkową z tego powodu przy dodawaniu tracimy mało istotne bity czego wynikiem jest mało dokładny wynik

In [61]:
import numpy as np
N = 100000000
C16 = np.float16(0.09531258654533566)
C32 = np.float32(0.09531258654533566)

sum16 = np.float16(0.0) 
for i in range(0, N):
    sum16 += C16

sum32 = np.float32(0.0)
for i in range(0, N):
    sum32 += C32
    
print(sum16, sum32)
print(N*C)

256.0 2097152.0
9533691.40625


Jak widać wynik dla typów o mniejszej precyzji jest o wiele mniej dokładny. <br> <br>
Poprawnym sposobem sumowania takich liczb jest algorytm sumacyjny Kahana. Uwzględnia on poprawke na utracone niskie bity w każdej iteracji, aby w następnej iteracji ją uwzględnić.

In [57]:
N = 100000000
C = 0.09531258654533566;
c = 0.0
sum = 0.0
for i in range(0, N):
  y = C - c
  t = sum + y
  c = (t - sum) - y
  sum = t

print(sum)
print(N*C)


9531258.654533565
9531258.654533565
